In [ ]:
import zipfile
from pathlib import Path

from tqdm import tqdm

import deepecohab

#### Make project

In [ ]:
example_dir = Path.cwd()

with zipfile.ZipFile(example_dir / "test_data_field" / "data.zip", "r") as zip_ref:
	zip_ref.extractall(example_dir / "test_data_field")

In [ ]:
config_path = deepecohab.create_ecohab_project(
	project_location=example_dir,  # Location where the project will be created
	experiment_name="test_field2",  # Name of the project
	data_path=example_dir / "test_data_field",  # Folder with ecohab txt files
	light_phase_start="07:00:00",
	dark_phase_start="20:00:00",
	field_ecohab=True,
	animal_ids=[
		"18D8E51A04",
		"2875E61A04",
		"2E749C1A04",
		"6AE5FF1904",
		"701EA81904",
		"71DFFF1904",
		"76F1E51A04",
		"815DE61A04",
		"8841E61A04",
		"9A44E61A04",
		"A43FE61A04",
		"B123A81904",
		"BEE7FF1904",
		"D16FA81904",
		"EB1FA81904",
	],
)

#### Field Eco-Hab layout

This image represents the layout of the cages and antennas:

<img src="../examples\test_data_field\img.png" alt="drawing" width="800"/>

#### Make structure

In [ ]:
lf = deepecohab.get_ecohab_data_structure(
	config_path,
	fname_prefix="COM",
	sanitize_animal_ids=True,
	min_antenna_crossings=100,
	timezone="Europe/Warsaw",
	custom_layout=True, # compatibility for antenna renaming
)

#### Run antenna analysis pipeline

In [ ]:
pipeline = deepecohab.df_registry.run_pipeline(
	config_path,
	minimum_time=2,
	chasing_time_window=[0.1, 1.2],
)

with tqdm(total=len(deepecohab.df_registry.analysis_steps)) as pbar:
	for step_name, _, _ in pipeline:
		pbar.set_description(f"Processing {step_name}")
		pbar.update(1)

print("Done!")

### Load data you're interested in

In [ ]:
deepecohab.df_registry.list_available()

In [ ]:
chasings_df = deepecohab.load_ecohab_data(config_path, 'chasings_df', return_df=True)

### Generate plots in the notebook

In [ ]:
deepecohab.plot_registry.list_available()

In [ ]:
cfg = deepecohab.read_config(config_path)
plt_cfg = deepecohab.PlotConfig(
    store = {
        "chasings_df": chasings_df,
    },
    days_range=[2,5],
    agg_switch='sum',
    animals=cfg['animal_ids'],
    animal_colors=deepecohab.utils.auxfun_plots.color_sampling(cfg['animal_ids']),
)

deepecohab.plot_registry.get_plot("chasings-line", plt_cfg)[0]